In [2]:
import streamlit as st
import openai
import re

# Set up OpenAI API key
openai.api_key = "YOUR API"

# Create a multi-line text input for code
code = st.text_area("Enter your code here:", height=200)
query_code=""

# Create a file uploader that only accepts code files
uploaded_file = st.file_uploader("Upload a file",
                                 type=['.py', '.js', '.cpp', '.h', '.cs', '.java', '.rb', '.php', '.go', '.swift',
                                       '.ts', '.sql', '.sh', '.html', '.htm', '.css', '.xml', '.json']
                                )
query_file=""

# Check if the text contains common code patterns
if code:
    query_code = "is there any vulnerability in the code given"+"\n"+code
    lines = code.split("\n")
    # Split the text into lines
    valid = False
    for line in lines:
        # Process each line of text
        if re.search(r"\b(def|class|import|print)\b", line) or \
            re.search(r"[()\[\];{},]", line):
            valid = True
        else:
            valid = False
            break
    if valid:
        st.success("Your input appears to be valid code!")
    else:
        st.error("Your input does not appear to be valid code.")
        
        
# Check if a file was uploaded

if uploaded_file is not None:
    # Read the contents of the file
    file_contents = uploaded_file.read().decode("utf-8")
    # Display the contents of the file
    st.write("File contents:")
    st.code(file_contents)
    query_file = "is there any vulnerability in the code given"+"\n"+file_contents
    
if query_code or query_file:
    response = openai.Completion.create(
        engine='text-davinci-003',
        prompt=query_code or query_file,
        max_tokens=1000,
        n=1,
        stop=None,
        temperature=0.5,
    )
    result = response.choices[0].text
    # Display the result
    st.write("Response:")
    st.write(result)
    
    # Continue the conversation based on the response
    counter = 0
    while True:
        counter += 1
        continuation = st.text_input("Enter your response:", key=counter)
        if continuation:
            query = result.strip() + "\n" + continuation.strip()
            response = openai.Completion.create(
                engine='text-davinci-003',
                prompt=query,
                max_tokens=1000,
                n=1,
                stop=None,
                temperature=0.5,
            )
            result = response.choices[0].text
            # Display the result
            st.write("Response:")
            st.write(result)
        else:
            break